# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 03**: Data Cleaning and Transformation Pipeline

**Date**: September 18th 2025

**Student Name**: Santiago Montes de Oca

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from santiagomdeo.spark_utils import proof
from santiagomdeo.spark_utils import SparkUtils

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on SparkSQL") \
    .master("spark://e52511a8c7e2:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/21 21:56:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
airlines_schema_columns = [("index", "int"), 
     ("airline", "string"), 
     ("flight", "string"),
     ("source_city", "string"),
     ("departure_time", "string"),
     ("stops", "string"),
     ("arrival_time", "string"),
     ("destination_city", "string"),
     ("class", "string"),
     ("duration", "float"),
     ("days_left", "int"),
     ("price", "int")
     ]
airlines_schema = SparkUtils.generate_schema(airlines_schema_columns)
airlines_schema

int
string
string
string
string
string
string
string
string
float
int
int


StructType([StructField('index', IntegerType(), True), StructField('airline', StringType(), True), StructField('flight', StringType(), True), StructField('source_city', StringType(), True), StructField('departure_time', StringType(), True), StructField('stops', StringType(), True), StructField('arrival_time', StringType(), True), StructField('destination_city', StringType(), True), StructField('class', StringType(), True), StructField('duration', FloatType(), True), StructField('days_left', IntegerType(), True), StructField('price', IntegerType(), True)])

In [3]:
df_airlines = spark.read \
                .option("header", "true") \
                .schema(airlines_schema) \
                .csv("/opt/spark/work-dir/data/airline/")

df_airlines.show(n=5)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|
|    3| Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|
|    4| Vistara| UK-963|      Delhi|       Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5955|
+-----+--------+-------+

## Drop unnecessary columns. Count how many null values the dataset has before/after the cleaning process
basically remove nulls

In [9]:
print(df_airlines.columns[1:-1])

['airline', 'flight', 'source_city', 'departure_time', 'stops', 'arrival_time', 'destination_city', 'class', 'duration', 'days_left']


In [10]:
from pyspark.sql.functions import col,isnan, when, count

df2 = df_airlines.dropna(subset=df_airlines.columns[1:-1])

In [14]:
print(df_airlines.count())
print(df2.count())

300153


300153


## Normalize categorical values: map “zero” → 0, “one” → 1, etc. in stops.
we will first see how many unique elements are in this set, and then map each one.

In [15]:
df2.select('stops').distinct().collect()

[Row(stops='two_or_more'), Row(stops='one'), Row(stops='zero')]

this really confuses me, cause if i have 2 or more, and the teacher kinda shows that he wants this data to be saved with integers instead of strings, so imma do both just in case

In [21]:
# the question with this, is that it probably creates teh schema for the new column as string, which would be anoying
df3 = df2.withColumn("Normalize Stops",
    when(col("stops") == 'one', 1)
    .otherwise(when(col('stops') == 'zero', 0)
               .otherwise(2)      ))

In [17]:
df3.show(n=5)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+---------------+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|Normalize Stops|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+---------------+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|              0|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|              0|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|              0|
|    3| Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          Mumbai|Economy|    2.25|        1| 5955|              0|
|    4| Vistara| UK-963|      Delh

## Create a new column called route: “Delhi → Mumbai” from source_city and destination_city.

In [34]:
from pyspark.sql.functions import lit, concat

df4 = df3.withColumn("route", concat(col("source_city"), lit("-->"), col("destination_city")))
df4.show(n=5)

+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+---------------+--------------+
|index| airline| flight|source_city|departure_time|stops| arrival_time|destination_city|  class|duration|days_left|price|Normalize Stops|         route|
+-----+--------+-------+-----------+--------------+-----+-------------+----------------+-------+--------+---------+-----+---------------+--------------+
|    0|SpiceJet|SG-8709|      Delhi|       Evening| zero|        Night|          Mumbai|Economy|    2.17|        1| 5953|              0|Delhi-->Mumbai|
|    1|SpiceJet|SG-8157|      Delhi| Early_Morning| zero|      Morning|          Mumbai|Economy|    2.33|        1| 5953|              0|Delhi-->Mumbai|
|    2| AirAsia| I5-764|      Delhi| Early_Morning| zero|Early_Morning|          Mumbai|Economy|    2.17|        1| 5956|              0|Delhi-->Mumbai|
|    3| Vistara| UK-995|      Delhi|       Morning| zero|    Afternoon|          M

## Transform departure_time and arrival_time to numerical categories (Morning, Afternoon, etc.), then encode as numbers (0=Early_Morning, 1=Morning, etc.).


In [39]:
df4.select('arrival_time').distinct().collect()

[Row(arrival_time='Evening'),
 Row(arrival_time='Morning'),
 Row(arrival_time='Late_Night'),
 Row(arrival_time='Afternoon'),
 Row(arrival_time='Early_Morning'),
 Row(arrival_time='Night')]

In [40]:
df4.select('departure_time').distinct().collect()

[Row(departure_time='Evening'),
 Row(departure_time='Morning'),
 Row(departure_time='Late_Night'),
 Row(departure_time='Afternoon'),
 Row(departure_time='Early_Morning'),
 Row(departure_time='Night')]

In [42]:
# Evening= 0, Morning= 1, Late_Night= 2, Afternoon= 3, Early_Morning= 4, Night= 5,

df5 = df4.withColumn(
"Num_Cat_Arrival_time",
when(col("arrival_time") == "Evening", 0)
.when(col("arrival_time") == "Morning", 1)
.when(col("arrival_time") == "Late_Night", 2)
.when(col("arrival_time") == "Afternoon", 3)
.when(col("arrival_time") == "Early_Morning", 4)
.when(col("arrival_time") == "Night", 5)
.otherwise("unknown")
) 
df6 = df5.withColumn(
"Num_Cat_Departure_time",
when(col("departure_time") == "Evening", 0)
.when(col("departure_time") == "Morning", 1)
.when(col("departure_time") == "Late_Night", 2)
.when(col("departure_time") == "Afternoon", 3)
.when(col("departure_time") == "Early_Morning", 4)
.when(col("departure_time") == "Night", 5)
.otherwise("unknown")
) 

In [44]:
df6.select("arrival_time","Num_Cat_Arrival_time","departure_time","Num_Cat_Departure_time" ).show(n=7)

+-------------+--------------------+--------------+----------------------+
| arrival_time|Num_Cat_Arrival_time|departure_time|Num_Cat_Departure_time|
+-------------+--------------------+--------------+----------------------+
|        Night|                   5|       Evening|                     0|
|      Morning|                   1| Early_Morning|                     4|
|Early_Morning|                   4| Early_Morning|                     4|
|    Afternoon|                   3|       Morning|                     1|
|      Morning|                   1|       Morning|                     1|
|    Afternoon|                   3|       Morning|                     1|
|      Morning|                   1|       Morning|                     1|
+-------------+--------------------+--------------+----------------------+
only showing top 7 rows


## Add a new column is_expensive: when(price > 6000, True).otherwise(False).

In [47]:
df7 = df6.withColumn("is_expensive", 
        when(col("price") > 6000, lit(True)).otherwise(lit(False))
    )

In [50]:
df7.select("price", "is_expensive").show(n=9)

+-----+------------+
|price|is_expensive|
+-----+------------+
| 5953|       false|
| 5953|       false|
| 5956|       false|
| 5955|       false|
| 5955|       false|
| 5955|       false|
| 6060|        true|
| 6060|        true|
| 5954|       false|
+-----+------------+
only showing top 9 rows


# In addition, the Notebook should also contain the results of the following aggregations:

### Get the average price per airline.



In [54]:
from pyspark.sql.functions import mean
df7.select(mean("price")).show(truncate=False)

+------------------+
|avg(price)        |
+------------------+
|20889.660523133203|
+------------------+



### Average duration per route

In [55]:
from pyspark.sql.functions import mean
df7.select(mean("duration")).show(truncate=False)

+-----------------+
|avg(duration)    |
+-----------------+
|12.22102081367383|
+-----------------+



### Minimum and maximum price per airline.

In [58]:
from pyspark.sql.functions import min, max
df7.select(min("price")).show(truncate=False)
df7.select(max("price")).show(truncate=False)

+----------+
|min(price)|
+----------+
|1105      |
+----------+



+----------+
|max(price)|
+----------+
|123071    |
+----------+



### Count flights by departure_time category

In [67]:
df7.select("departure_time").where(df7.departure_time==lit("Evening")).count()

65102

In [ ]:
df7.select("departure_time").where(df7.departure_time==lit("Morning")).count()
# Late_Night", 2)
# .when(col("departure_time") == "Afternoon", 3)
# .when(col("departure_time") == "Early_Morning", 4)
# .when(col("departure_time") == "Night", 5)

71146

In [69]:
df7.select("departure_time").where(df7.departure_time==lit("Afternoon")).count()

47794